In [29]:
# Create transferfunction!
def transferfunction(freq): #Where B is given, NOT H!
    mu=4*np.pi*10**(-7)
    if freq<=0:
        raise Exception('Frequency cannot be any lower or equal to zero!')
    if freq<=10**-5:
        Zn=0
    else:
        # conductivity parameters
        if model == 1:
            cond=np.zeros((2,4)) #business as usual
            cond[0,:]=[1./1.,1./5000.,1./1.,1./10.] #conductivity top to bottom
            cond[1,:]=[2000,6000,4000,0] #depth top to bottom in m
        elif model == 2:
            cond=np.zeros((2,4)) #deep ocean
            cond[0,:]=[4.,1./5000.,1./1.,1./10.] #conductivity top to bottom
            cond[1,:]=[2000,6000,4000,0] #depth top to bottom in m
        elif model == 3:
            cond=np.zeros((2,2)) #GIC in Europe paper
            cond[0,:]=[1./38.5,1./0.385] #conductivity top to bottom
            cond[1,:]=[150000,0] #depth top to bottom in m
        elif model == 4:
            cond=np.zeros((2,4)) #MODELLING OCEAN EFFECT IN LOCAL C-RESPONSES: oceanic mantle
            cond[0,:]=[1./10**3,1./20.,1./2.,1./0.42] #conductivity top to bottom
            cond[1,:]=[100000,400000,200000,0] #depth top to bottom in m
        elif model == 5:
            cond=np.zeros((2,4)) #MODELLING OCEAN EFFECT IN LOCAL C0-RESPONSES: continental mantle
            cond[0,:]=[1./(3*10**3),1./70.,1./16.,1./0.42] #conductivity top to bottom
            cond[1,:]=[100000,400000,200000,0] #depth top to bottom in m
        elif model == 6: #Pirjola et al 2014: Geomagnetically induced currents in Europe 
            cond=np.zeros((2,5))
            cond[0,:]=[1./(40),1./3.,1./2000.,1./118., 1/15.] #conductivity top to bottom
            cond[1,:]=[400,1300,140000,170000,0] #depth top to bottom in m
        elif model == 7: # combi model 
            cond=np.zeros((2,7))
            cond[0,:]=[1.,1./5000.,1.,1./(3*10**3),1/70.,1/16.,1/0.42] #conductivity top to bottom
            cond[1,:]=[2000,6000,4000,88000,400000,200000,0] #depth top to bottom in m
        else:
            cond=np.zeros((2,5)) #bit of water (50m)
            cond[0,:]=[4,1./1.,1./5000.,1./1.,1./10.] #conductivity top to bottom
            cond[1,:]=[50,2000,6000,4000,0] #depth top to bottom in m
        #first do bottom layer
        kn=np.sqrt(1j*freq*mu*cond[0,-1])
        Zn=1j*freq*mu/kn

        # iterate from bottom to top
        for item in range(2,len(cond[0])+1): #we go in opposite direction later, see Trichtchenko and Boteler (2002)
            kn=np.sqrt(1j*freq*mu*cond[0,-item])
            rn=(1-kn*(Zn/(1j*freq*mu)))/(1+kn*(Zn/(1j*freq*mu)))
            Zn=1j*freq*mu*((1-rn*np.exp(-2*kn*cond[1,-item]))/(kn*(1+rn*np.exp(-2*kn*cond[1,-item]))))

    return Zn/mu

In [3]:
######################## writing results ###########################################
def Vec_electric(thread,path,ElectricN,ElectricE,begin,end,lon,lat):
    for localvar.item in range(begin,end):
        logging.info(f'Thread {thread} is writing step {localvar.item}.')
        scaling=10**-4
        localvar.correction=np.zeros(len(ElectricN[0]))
        for localvar.counter,localvar.item2 in enumerate(ElectricN[localvar.item,:]):
            if localvar.item2<0:
                localvar.correction[localvar.counter]=180
            else:
                localvar.correction[localvar.counter]=0

        localvar.newfile=pd.DataFrame(columns=['lon','lat','heading','length'])
        localvar.newfile.at[:,'lon']=lon
        localvar.newfile.at[:,'lat']=lat
        localvar.newfile.at[:,'heading']=np.degrees(np.arctan(ElectricE[localvar.item,:]/ElectricN[localvar.item,:]))+localvar.correction[:]
        localvar.newfile.at[:,'length']=np.sqrt(ElectricE[localvar.item,:]**2+ElectricN[localvar.item,:]**2)/scaling

        if localvar.item<10:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_000{localvar.item}.csv', sep=' ', index=False, header=False)
        if localvar.item<100 and localvar.item>9:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_00{localvar.item}.csv', sep=' ', index=False, header=False)
        if localvar.item<1000 and localvar.item>99:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_0{localvar.item}.csv', sep=' ', index=False, header=False)
        if localvar.item>999:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_{localvar.item}.csv', sep=' ', index=False, header=False)
        

In [4]:
######################## writing results ###########################################
def writing_electric(thread,path,Electric,begin,end,lon,lat):
    for localvar.item in range(begin,end):
        logging.info(f'Thread {thread} is writing step {localvar.item}.')
        localvar.newfile=pd.DataFrame(columns=['lon','lat','value'])
        localvar.newfile.at[:,'lon']=lon
        localvar.newfile.at[:,'lat']=lat
        localvar.newfile.at[:,'value']=Electric[localvar.item,:]

        if localvar.item<10:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_000{localvar.item}.csv', sep=' ', index=False, header=False)
        if localvar.item<100 and localvar.item>9:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_00{localvar.item}.csv', sep=' ', index=False, header=False)
        if localvar.item<1000 and localvar.item>99:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_0{localvar.item}.csv', sep=' ', index=False, header=False)
        if localvar.item>999:
            localvar.newfile.to_csv(path_or_buf=f'{path}/electric_{localvar.item}.csv', sep=' ', index=False, header=False)
        

In [30]:
import scipy.signal.windows as spsw
import numpy as np
import os
from multiprocessing import Process
import pandas as pd
from time import process_time
from threading import local
import logging
localvar=local()
path='/nobackup/users/out/Magnetic_field/Halloween'

# import magnetic field data in X/Y-direction (north)
magnetic_Xfiles=[]
magnetic_Yfiles=[]

############################# get the strings ###################################
os.system(f"ls {path}/interpolation/minute_????.csv > {path}/tempX.txt")
os.system(f"ls {path}/interpolation/minute_????.csv.Y > {path}/tempY.txt")
f=open(f'{path}/tempX.txt')
for item in f:
    item=item.strip('\n')
    magnetic_Xfiles.append(item)
f.close()
os.system(f'rm {path}/tempX.txt')
f=open(f'{path}/tempY.txt')
for item in f:
    item=item.strip('\n')
    magnetic_Yfiles.append(item)
f.close()
os.system(f'rm {path}/tempY.txt')
   
magnetic_Xfiles=sorted(magnetic_Xfiles) #sort to number 0000-1440
magnetic_Yfiles=sorted(magnetic_Yfiles)

for file in magnetic_Xfiles:
    Xfile=pd.read_csv(file, delimiter=' ', header=None)
    break
for file in magnetic_Yfiles:
    Yfile=pd.read_csv(file, delimiter=' ', header=None)
    break

lat=np.zeros(len(Xfile))
lon=np.zeros(len(Xfile))
MX_matrix=np.zeros((len(magnetic_Xfiles),len(Xfile))) #matrix for storing values (vertical same place, horizontal same time)
MXft_matrix=np.zeros((int(len(magnetic_Xfiles)/2)+1,len(Xfile)),dtype='complex')
EX_matrix=np.zeros((len(magnetic_Yfiles),len(Yfile))) 
EXft_matrix=np.zeros((int(len(magnetic_Yfiles)/2)+1,len(Yfile)),dtype='complex')
MY_matrix=np.zeros((len(magnetic_Yfiles),len(Yfile))) #matrix for storing values (vertical same place, horizontal same time)
MYft_matrix=np.zeros((int(len(magnetic_Yfiles)/2)+1,len(Yfile)),dtype='complex')
EY_matrix=np.zeros((len(magnetic_Xfiles),len(Xfile))) 
EYft_matrix=np.zeros((int(len(magnetic_Xfiles)/2)+1,len(Xfile)),dtype='complex')
################################################################################# 
########################### get the values ######################################
######################### first x-direction #####################################
print('hi, we are still working!')
# t1_start=process_time()
for counter,file in enumerate(magnetic_Xfiles):
    Xfile=pd.read_csv(file, delimiter=' ', header=None)
    values=Xfile.to_numpy()
    MX_matrix[counter,:]=values[:,2]/(10**9)*1 #scaling factor
lat=values[:,1]
lon=values[:,0]
for counter,file in enumerate(magnetic_Yfiles):
    Yfile=pd.read_csv(file, delimiter=' ', header=None)
    values=Yfile.to_numpy()
    MY_matrix[counter,:]=values[:,2]/(10**9)*1

# t1_end=process_time()
# print(f'elapsed time for reading files is {t1_end-t1_start} seconds')
# print(MX_matrix)
# print(MY_matrix)

############## start fourier transformation ######################
print('hihi')
# t2_start=process_time()
for column in range(len(MX_matrix[0])):
    MXft_matrix[:,column]=np.fft.rfft(MX_matrix[:,column]*spsw.hann(len(MX_matrix))) #multiply with hanning window to reduce edge effects
for column in range(len(MY_matrix[0])):
    MYft_matrix[:,column]=np.fft.rfft(MY_matrix[:,column]*spsw.hann(len(MY_matrix)))
    # use rfft to only retain positive frequencies
    
# t2_end=process_time()
# print(f'elapsed time for calculating fourier transform is {t2_end-t2_start} seconds')
# print(MXft_matrix)
# print(MYft_matrix)

######################### calculate Electric field in frequency direction #############################3
# make frequencyvector in seconds
df=1./(24*60*60*3.) # seconds! #aangepast
fmax=1./(2*60.)
freqvec=np.arange(0,fmax+df,df) 

# t3_start=process_time() #1d conductivity model!
for row in range(1,len(MXft_matrix)): #zero is not allowed, same row = same frequency
    EYft_matrix[row,:]=-1*MXft_matrix[row,:]*transferfunction(freqvec[row])
for row in range(1,len(MYft_matrix)): #zero is not allowed
    EXft_matrix[row,:]=MYft_matrix[row,:]*transferfunction(freqvec[row])

# t3_end=process_time()
# print(f'elapsed time for applying transferfunction is {t3_end-t3_start} seconds')
# print(EYft_matrix)
# print(EXft_matrix)

######################## fourier transform back ####################################
# t4_start=process_time()
for column in range(len(EYft_matrix[0])):
    EY_matrix[:,column]=np.fft.irfft(EYft_matrix[:,column])
for column in range(len(EXft_matrix[0])):
    EX_matrix[:,column]=np.fft.irfft(EXft_matrix[:,column])

# t4_end=process_time()
# print(f'elapsed time for backward fourier transforming is {t4_end-t4_start} seconds')
# print(EY_matrix)
# print(EX_matrix)

######################### writing E field to files #################################
# t5_start=process_time()
# path='/nobackup/users/out/Magnetic_field/Halloweenx10'
# logging.basicConfig(filename=f'{path}/logbookelectric.log', level=logging.DEBUG, format='%(asctime)s %(message)s')
# try:
#     os.mkdir(f'{path}/electric_field_east')
# except:
#     logging.warning('Directory is already created, data could be overwritten.')
# try:
#     os.mkdir(f'{path}/electric_field_north')
# except:
#     logging.warning('Directory is already created, data could be overwritten.')

# n=6
# nrsteps=int(1440*3/n) #aangepast
# threads=list()
# for index in range(n):
#     q=Process(target=writing_electric, args=(index+1, f'{path}/electric_field_east', EY_matrix, nrsteps*index, nrsteps*(index+1), lon, lat))
#     threads.append(q)
#     q.start()
# for thread in threads:
#     thread.join()

# threads=list()
# for index in range(n):
#     q=Process(target=writing_electric, args=(index+1, f'{path}/electric_field_north', EX_matrix, nrsteps*index, nrsteps*(index+1), lon, lat))
#     threads.append(q)
#     q.start()
# for thread in threads:
#     thread.join()
# t5_end=process_time()
# print(f'elapsed time for writing is {t5_end-t5_start} seconds')

# try:
#     os.mkdir(f'{path}/electric_field_horizontal')
# except:
#     logging.warning('Directory is already created, data could be overwritten.')

# n=6
# nrsteps=int(1440/n)
# threads=list()
# for index in range(n):
#     q=Process(target=Vec_electric, args=(index+1, f'{path}/electric_field_horizontal', EX_matrix, EY_matrix, nrsteps*index, nrsteps*(index+1), lon, lat))
#     threads.append(q)
#     q.start()
# for thread in threads:
#     thread.join()

hi, we are still working!
hihi


In [31]:
print(EX_matrixorg)
print(EX_matrix)

[[8.06157814e-08 8.11855425e-08 8.17135691e-08 ... 1.33300632e-07
  1.34000263e-07 1.34690944e-07]
 [7.58803594e-08 7.64763063e-08 7.70339634e-08 ... 1.14812581e-07
  1.15558268e-07 1.16295517e-07]
 [8.00386457e-08 8.06041675e-08 8.11281457e-08 ... 1.32640093e-07
  1.33330692e-07 1.34012378e-07]
 ...
 [7.70725429e-08 7.76770410e-08 7.82428889e-08 ... 1.16133912e-07
  1.16897998e-07 1.17653579e-07]
 [8.11816466e-08 8.17555916e-08 8.22876160e-08 ... 1.33945178e-07
  1.34653766e-07 1.35353370e-07]
 [7.64804377e-08 7.70806837e-08 7.76424579e-08 ... 1.15479327e-07
  1.16234248e-07 1.16980698e-07]]
[[8.04848291e-08 8.10557223e-08 8.15850381e-08 ... 1.32576543e-07
  1.33279359e-07 1.33973253e-07]
 [7.60876712e-08 7.66826088e-08 7.72391587e-08 ... 1.15468388e-07
  1.16213432e-07 1.16950028e-07]
 [7.99085880e-08 8.04752559e-08 8.10005362e-08 ... 1.31918068e-07
  1.32611874e-07 1.33296794e-07]
 ...
 [7.72742850e-08 7.78777333e-08 7.84424349e-08 ... 1.16784229e-07
  1.17547596e-07 1.18302449e-07]